Imports:

In [19]:
import random
import gymnasium as gym
from gymnasium import spaces
import numpy as np


### **General Unit class**

In [20]:
class Unit:
  """
    General unit class.
  """
  def __init__(self, unit_name: str, unit_type: str, cost: int, star: int = 1):
    self.unit_name = unit_name
    self.unit_type = unit_type # We have three types of units: 1) "Healer", 2) "Tank", 3) "Ranger".
    self.cost = cost
    self.star = star  # Level of a unit: 1, 2, or 3.
    self.hp = self.get_hp()
    self.damage = self.get_damage()
    self.ability_ready = False
    self.attack_counter = 0 # After unit attacks 5 times - it can use ability.

  def get_hp(self):
    """
      Returns hp of a unit.
    """
    # Dictionary of hp values for 1-star units
    hp_values_1star = {
        1: 500,
        2: 650,
        3: 800,
        4: 950,
        5: 1100
    }

    # Dictionary of multipliers for increasing hp based on level
    star_multipliers = {
        1: 1.0,
        2: 1.5, # 50% increase
        3: 2 # 100% increase
    }

    # Dictionary of unit types for increasing hp based on type
    unit_type = {
        "Tank" : 1.4, # 40% increase
        "Ranger": 0.8, # 20% decrease
        "Healer": 0.9 # 10% decrease
    }

    # Combine the guidelines from above to calculate the hp value
    hp = int(hp_values_1star[self.cost] * star_multipliers[self.star] * unit_type[self.unit_type])
    #hp = int(star_multipliers[self.star] * unit_type[self.unit_type])

    return hp

  def get_damage(self):
    """
      Returns damage of a unit.
    """

    # Dictionary of damage values for 1-star units
    damage_values_1star = {
        1: 70,
        2: 80,
        3: 90,
        4: 100,
        5: 120
    }

    # Dictionary of multipliers for increasing damage based on level
    star_multipliers = {
        1: 1.0,
        2: 2, # 100% increase
        3: 3 # 200% increase
    }

    # Dictionary of unit types for increasing damage based on type
    unit_type = {
        "Tank" : 0.7, # 30% decrease
        "Ranger": 1.5, # 50% increase
        "Healer": 0.9 # 10% decrease
    }

    # Combine the guidelines from above to calculate the damage value
    damage = int(damage_values_1star[self.cost] * star_multipliers[self.star] * unit_type[self.unit_type])
    #damage = int(star_multipliers[self.star] * unit_type[self.unit_type])

    return damage

  def attack(self):
    """
      Stacks attacks counter to gain ability.
    """
    self.attack_counter += 1
    if self.attack_counter == 5:
      self.ability_ready = True

  def reset_ability_charge(self):
    """
      Resets the ability.
    """
    self.attack_counter = 0
    self.ability_ready = False

  def __repr__(self):
    """
      Returns a string representation of the unit (debugging purposes).
    """
    return f"<{self.star}★ {self.unit_name}, type:{self.unit_type} ({self.cost}-cost) - {self.hp} HP, {self.damage} damage>"

In [21]:
all_units_names_role_and_cost = {
    # 1 cost
    "Silent": ("Ranger", 1), "Flamy": ("Ranger", 1), "Cheddy": ("Ranger", 1), "Hertrude": ("Ranger", 1),
    "Brim": ("Tank", 1), "Bravos": ("Tank", 1), "Lorak": ("Tank", 1), "Kiros": ("Tank", 1),
    "Mary": ("Healer", 1), "Looney": ("Healer", 1), "Kitana": ("Healer", 1), "Miss Luis": ("Healer", 1),
    # 2 cost
    "Marko": ("Ranger", 2), "Colt": ("Ranger", 2), "Kana": ("Ranger", 2),
    "Morfus": ("Tank", 2), "Sol": ("Tank", 2), "Kemer": ("Tank", 2), "Pronto": ("Tank", 2),
    "Summer": ("Healer", 2), "Clover": ("Healer", 2), "Pishta": ("Healer", 2),
    # 3 cost
    "Bruno": ("Ranger", 3), "Tofa": ("Ranger", 3), "Monroe": ("Ranger", 3),
    "Krusty": ("Tank", 3), "Kenny": ("Tank", 3), "Kanye": ("Tank", 3),
    "Ashley": ("Healer", 3), "Bonny": ("Healer", 3),
    # 4 cost
    "Kaneki Ken": ("Ranger", 4), "Satoru Gojo": ("Ranger", 4), "Gabimaru": ("Ranger", 4),
    "Toochka": ("Tank", 4), "MnSano": ("Tank", 4),
    "Avotushenka": ("Healer", 4),
    # 5 cost
    "Keysella": ("Ranger", 5),
    "Maikeru": ("Tank", 5),
    "Militmi": ("Healer", 5)
}

all_units_list = []

for name, (role, cost) in all_units_names_role_and_cost.items():
  all_units_list.append(Unit(name, role, cost))

for unit in all_units_list:
  print(unit)

<1★ Silent, type:Ranger (1-cost) - 400 HP, 105 damage>
<1★ Flamy, type:Ranger (1-cost) - 400 HP, 105 damage>
<1★ Cheddy, type:Ranger (1-cost) - 400 HP, 105 damage>
<1★ Hertrude, type:Ranger (1-cost) - 400 HP, 105 damage>
<1★ Brim, type:Tank (1-cost) - 700 HP, 49 damage>
<1★ Bravos, type:Tank (1-cost) - 700 HP, 49 damage>
<1★ Lorak, type:Tank (1-cost) - 700 HP, 49 damage>
<1★ Kiros, type:Tank (1-cost) - 700 HP, 49 damage>
<1★ Mary, type:Healer (1-cost) - 450 HP, 63 damage>
<1★ Looney, type:Healer (1-cost) - 450 HP, 63 damage>
<1★ Kitana, type:Healer (1-cost) - 450 HP, 63 damage>
<1★ Miss Luis, type:Healer (1-cost) - 450 HP, 63 damage>
<1★ Marko, type:Ranger (2-cost) - 520 HP, 120 damage>
<1★ Colt, type:Ranger (2-cost) - 520 HP, 120 damage>
<1★ Kana, type:Ranger (2-cost) - 520 HP, 120 damage>
<1★ Morfus, type:Tank (2-cost) - 909 HP, 56 damage>
<1★ Sol, type:Tank (2-cost) - 909 HP, 56 damage>
<1★ Kemer, type:Tank (2-cost) - 909 HP, 56 damage>
<1★ Pronto, type:Tank (2-cost) - 909 HP, 56 da

In [22]:
# Tests
lilBabyRanger = Unit("Silent", "Ranger", 5, 3)
print(lilBabyRanger)

<3★ Silent, type:Ranger (5-cost) - 1760 HP, 540 damage>


### **Player Class 😎**

In [23]:
class Player:
  """
    General player class.
  """
  def __init__(self, name: str):
    self.name = name
    # Starting gold, level, hp, and no units for every player.
    self.gold = 13
    self.level = 3
    self.max_units_on_board = self.level
    self.hp = 100
    self.board = [[None for _ in range(8)] for _ in range(4)]
    self.bench = [None for _ in range(8)]
    self.all_units = []
    self.shop = Shop(all_units_list, self.level)
    self.won_last_fight = False

  def gain_gold(self):
    """
      Gain gold - method that triggers every start of the round. Player gets:
        1) + win bonus if they won last fight;
        2) + interest rate (no more than 5);
        3) + 9 gold.
    """
    # Win bonus.
    win_bonus = 0
    if self.won_last_fight:
      win_bonus = 1

    # Interest rate.
    interest_rate = self.gold // 10
    if interest_rate > 5:
      interest_rate = 5

    # Gaining gold.
    self.gold += 9 + win_bonus + interest_rate
  
  def refresh_shop(self):
    if self.gold >= 2:
      self.gold -= 2
      self.shop.update(self.level)

  def buy_unit_from_shop(self, unit):
    """
      Buy a unit from shop.
    """
    if unit is None:
      return False
    
    if self.gold < unit.cost:
      return False

    for i in range(8):
      if self.bench[i] is None:
        self.bench[i] = unit
        self.all_units.append(unit)
        self.gold -= unit.cost
        return True
    return False
  
  def buy_unit(self, shop_index):
    """
      Buy_unit_from_shop() function wrapper.
    """
    unit = self.shop.units_in_shop[shop_index]
    success = self.buy_unit_from_shop(unit)
    if success:
        self.shop.remove(unit)
    return success

  def sell_unit(self, unit):
    """
      Sells a unit.
    """
    self.all_units.remove(unit)
    self.gold += unit.cost * unit.star
    if unit in self.bench:
      for i in range(8):
        if self.bench[i] == unit:
          self.bench[i] = None
          break
    elif unit in self.board:
      for i in range(4):
        for j in range(8):
          if self.board[i][j] == unit:
            self.board[i][j] = None
            break

### **Shop Class**

In [24]:
# I just started it on random, you can change anything (Alex).
# I am working on all_units_lsit. Treat it as a list of units (objects of unit class).
class Shop:
  """
    General shop class.
  """
  def __init__(self, all_units_list, player_level):
    self.units_in_shop = [None] * 5 # List of the 5 units to choose from
    self.all_units_list = all_units_list # List of all available units
    self.fill_shop(player_level) # Fill the shop with units initially
  
  def update(self, player_level):
    """
    Updates (rerolls) the shop. Should be called if player pays 2 gold.
    """
    self.fill_shop(player_level)

  def fill_shop(self, player_level):
    """
      Fills the shop with units.
    """
    self.units_in_shop = [None] * 5 # Reset the shop
    probabilities = self.get_probabilities(player_level) # Get probabilities for each unit

    for i in range(5):
      roll = random.random() * 100
      cumulative_prob = 0
      selected_cost = 1

      for cost, prob in probabilities.items():
        cumulative_prob += prob
        if roll <= cumulative_prob:
          selected_cost = cost
          break
      cost_units = [unit for unit in self.all_units_list if unit.cost == selected_cost]

      # Select a random unit if available
      if cost_units:
        self.units_in_shop[i] = random.choice(cost_units)

  def get_probabilities(self, player_level):
    """
      Returns a list of probabilities for each unit in the shop.
    """
    # Probability distributions for each unit in the shop
    distributions = {
        3: {1: 75, 2: 25, 3: 0, 4: 0, 5: 0}, # Start with 3 level
        4: {1: 60, 2: 30, 3: 10, 4: 0, 5: 0},
        5: {1: 40, 2: 35, 3: 20, 4: 5, 5: 0},
        6: {1: 25, 2: 40, 3: 25, 4: 10, 5: 0},
        7: {1: 15, 2: 30, 3: 35, 4: 15, 5: 5},
        8: {1: 10, 2: 20, 3: 25, 4: 35, 5: 10},
        9: {1: 5, 2: 15, 3: 20, 4: 40, 5: 20}, # 9 levels max
    }

    return distributions[player_level]

  def remove(self, unit):
    """
      Removes a unit from the shop.
    """

    for i in range(len(self.units_in_shop)):
      if self.units_in_shop[i] == unit:
        self.units_in_shop[i] = None
        return True
    return False

  def __repr__(self):
    """
      Returns a string representation of the unit (debugging purposes).
    """
    unit_names = [str(unit) if unit else "Empty" for unit in self.units_in_shop]
    return f"Shop: {unit_names}"


In [25]:
# Testing shop working with a player.
player = Player("Alex")
print(player.gold)
print(player.shop)

13
Shop: ['<1★ Bravos, type:Tank (1-cost) - 700 HP, 49 damage>', '<1★ Sol, type:Tank (2-cost) - 909 HP, 56 damage>', '<1★ Pishta, type:Healer (2-cost) - 585 HP, 72 damage>', '<1★ Looney, type:Healer (1-cost) - 450 HP, 63 damage>', '<1★ Kitana, type:Healer (1-cost) - 450 HP, 63 damage>']


In [26]:
player.refresh_shop()

In [27]:
print(player.gold)
print(player.shop)

11
Shop: ['<1★ Pronto, type:Tank (2-cost) - 909 HP, 56 damage>', '<1★ Looney, type:Healer (1-cost) - 450 HP, 63 damage>', '<1★ Hertrude, type:Ranger (1-cost) - 400 HP, 105 damage>', '<1★ Flamy, type:Ranger (1-cost) - 400 HP, 105 damage>', '<1★ Lorak, type:Tank (1-cost) - 700 HP, 49 damage>']


In [28]:
player.buy_unit(0)

True

In [29]:
print(player.gold)
print(player.shop)
print(player.bench)

9
Shop: ['Empty', '<1★ Looney, type:Healer (1-cost) - 450 HP, 63 damage>', '<1★ Hertrude, type:Ranger (1-cost) - 400 HP, 105 damage>', '<1★ Flamy, type:Ranger (1-cost) - 400 HP, 105 damage>', '<1★ Lorak, type:Tank (1-cost) - 700 HP, 49 damage>']
[<1★ Pronto, type:Tank (2-cost) - 909 HP, 56 damage>, None, None, None, None, None, None, None]


### **Environment?**

In [30]:
player1 = Player("Player 1")
player2 = Player("Player 2")

In [31]:
END_TURN_ACTION = 7

class TFTEnv(gym.Env):
    def __init__(self):
        super().__init__()

        # Player initialization.
        self.player1 = player1
        self.player2 = player2
        self.current_player = self.player1

        # Maximum number of steps before fight.
        self.max_steps_per_round = 50
        self.steps_this_round = 0

        # TODO - I am not sure about it.
        self.action_space = spaces.Discrete(8)  # 0-4: buy a unit, 5: reroll shop, 6: level up, 7: END_TURN_ACTION
        self.observation_space = spaces.Dict({
            "gold": spaces.Box(low=0, high=np.inf, shape=(), dtype=np.float32),
            "health": spaces.Discrete(101),
            "shop": spaces.MultiDiscrete([6] * 5),
        })

        self.done = False

    def reset(self, seed=None, options=None):
        self.player1 = player1
        self.player2 = player2
        self.current_player = self.player1
        self.steps_this_round = 0
        self.done = False

        observation = self.get_observation()
        return observation, {}

    def step(self, action):
        """
            Make an action.
        """

        if self.done:
            raise Exception("Game is over. Call reset().")

        reward = 0

        # Actions:
        if action in range(5):  # Buy 1 out of 5 units from the shop.
            unit = self.current_player.shop.units_in_shop[action]
            if unit and self.current_player.gold >= unit.cost:
                self.current_player.buy_unit(action)  # Changed: pass action (index) instead of unit
                self.current_player.shop.remove(unit)
        elif action == 5:  # Reroll the shop.
            if self.current_player.gold >= 2:
                self.current_player.gold -= 2
                self.current_player.shop.update(self.current_player.level)
        elif action == 6:  # Level up.
            if self.current_player.gold >= 4:
                self.current_player.gold -= 4
                self.current_player.level += 1
        elif action == END_TURN_ACTION:  # End player's turn.
            self.start_fight()

        # Count steps in a round.
        self.steps_this_round += 1

        # Check if the fight can start.
        if self.steps_this_round >= self.max_steps_per_round:
            self.start_fight()

        # Check the final health.
        if self.player1.hp <= 0 or self.player2.hp <= 0:
            self.done = True

        observation = self.get_observation()
        return observation, reward, self.done, False, {}

    def start_fight(self):
        """
            Start the fight.
        """
        self.steps_this_round = 0  # Reset the counter.

        # Make random winner (for now).
        winner = random.choice([self.player1, self.player2])
        loser = self.player1 if winner == self.player2 else self.player2

        loser.hp -= 10  # Lose 10 hp for a loss.

    def get_observation(self):
        """
            Return current player's observation.
        """
        shop_obs = [0 if unit is None else unit.cost for unit in self.current_player.shop.units_in_shop]
        return {
            "gold": float(self.current_player.gold),
            "hp": self.current_player.hp,
            "shop": np.array(shop_obs, dtype=np.int64),
        }


In [32]:
import gymnasium as gym
env = TFTEnv()

obs, _ = env.reset()

done = False
while not done:
    action = env.action_space.sample()  # RANDOM ACTIONS (FOR NOW).
    obs, reward, done, truncated, info = env.step(action)
    print(obs)


{'gold': 12.0, 'hp': 100, 'shop': array([1, 1, 2, 0, 1])}
{'gold': 11.0, 'hp': 100, 'shop': array([1, 0, 2, 0, 1])}
{'gold': 11.0, 'hp': 100, 'shop': array([1, 0, 2, 0, 1])}
{'gold': 10.0, 'hp': 100, 'shop': array([0, 0, 2, 0, 1])}
{'gold': 10.0, 'hp': 100, 'shop': array([0, 0, 2, 0, 1])}
{'gold': 10.0, 'hp': 90, 'shop': array([0, 0, 2, 0, 1])}
{'gold': 10.0, 'hp': 90, 'shop': array([0, 0, 2, 0, 1])}
{'gold': 10.0, 'hp': 90, 'shop': array([0, 0, 2, 0, 1])}
{'gold': 6.0, 'hp': 90, 'shop': array([0, 0, 2, 0, 1])}
{'gold': 4.0, 'hp': 90, 'shop': array([0, 0, 0, 0, 1])}
{'gold': 4.0, 'hp': 90, 'shop': array([0, 0, 0, 0, 1])}
{'gold': 2.0, 'hp': 90, 'shop': array([1, 2, 1, 3, 1])}
{'gold': 0.0, 'hp': 90, 'shop': array([1, 2, 3, 2, 3])}
{'gold': 0.0, 'hp': 90, 'shop': array([1, 2, 3, 2, 3])}
{'gold': 0.0, 'hp': 90, 'shop': array([1, 2, 3, 2, 3])}
{'gold': 0.0, 'hp': 90, 'shop': array([1, 2, 3, 2, 3])}
{'gold': 0.0, 'hp': 90, 'shop': array([1, 2, 3, 2, 3])}
{'gold': 0.0, 'hp': 90, 'shop': arr